# Python API for KiFMM-rs

We provide a full, object oriented, Python API for our Rust library. Below are a number of examples of how to achieve different functionality.

In [1]:
%gui qt
import numpy as np
from mayavi import mlab
import matplotlib.pyplot as plt

from kifmm_py import (
    KiFmm,
    LaplaceKernel,
    HelmholtzKernel,
    SingleNodeTree,
    EvalType,
    BlasFieldTranslation,
    FftFieldTranslation,
)

********************************************************************************
         to build the TVTK classes (9.2). This may cause problems.
         Please rebuild TVTK.
********************************************************************************



## Setup

Setting up an FMM requires some coordinate data associated with source and target particles, which we expect as a flat Fortran ordered buffer of shape (n_coordinates, 3), where each column corresponds to the component of each axis.

Additionally the user needs to choose some basic parameters, principally

## Kernel

We support both `LaplaceKernel` and `HelmholtzKernel`, which has been is an active area of development for low-frequencies and for now offers no speed or convergence guarantees.

## Field Translation

Users must specify the acceleration schemes for the multipole to local (M2L) field translation step of the FMM.

We currently offer `FftFieldTranslation`, and `BlasFieldTranslation`, where the matrices associated with `BlasFieldTranslation` can optionally be compressed with a randomised SVD, or a deterministic SVD.

## Tree

Only `SingleNodeTree`s are provided at present, users can optionally discretise with a specified critical value of particles per leaf box `n_crit`, or by explicitly setting the tree's `depth`. In the latter case `expansion_order` must be set for each level of the tree, allowing for the option to set variable expansion order by level. This can be efficacious for the `HelmholtzKernel`.


## Basic Example (Laplace FMM + FFT M2L)

Here we use a FFT based M2L field translation, explicitly setting a block size for the field translation algorithm. We discretise the octree with an `n_crit` parameter, which defines the maximum number of particles per leaf box. Here we set `EvalType` to `Value`, meaning that we evaluate only potentials.

In [2]:
np.random.seed(0)

dim = 3
dtype = np.float32

# Set FMM Parameters
expansion_order = np.array([6], np.uint64)  # Single expansion order as using n_crit
n_vec = 1
n_crit = 150
n_sources = 1000
n_targets = 2000
prune_empty = True  # Optionally remove empty leaf boxes, their siblings, and ancestors, from the Tree

# Setup source/target/charge data in Fortran order
sources = np.random.rand(n_sources * dim).astype(dtype)
targets = np.random.rand(n_targets * dim).astype(dtype)
charges = np.random.rand(n_sources * n_vec).astype(dtype)

eval_type = EvalType.Value

# EvalType computes either potentials (EvalType.Value) or potentials + derivatives (EvalType.ValueDeriv)
kernel = LaplaceKernel(dtype, eval_type)

tree = SingleNodeTree(sources, targets, charges, n_crit=n_crit, prune_empty=prune_empty)

field_translation = FftFieldTranslation(kernel, block_size=32)

# Create FMM runtime object
fmm = KiFmm(expansion_order, tree, field_translation, timed=True)

# Evaluate potentials
fmm.evaluate()

# Examine potentials
fmm.all_potentials

# Examine operator times rounded in milliseconds
fmm.operator_times()

{'P2M': 0, 'M2M(2)': 0, 'M2M(1)': 0, 'M2L(2)': 2, 'P2P': 0, 'L2P': 0}

In [3]:
fmm.metadata_times()

{'source_data': 4, 'target_data': 3, 'source_to_target_data': 256}

In [4]:
fmm.communication_times()

{'source_domain': 0, 'target_domain': 0, 'source_tree': 0, 'target_tree': 0}

## Compare with Direct Evaluation

Note, as a part of the FMM input coordinate data is sorted into a `Morton' defined order, as a part of tree construction. Therefore to compare with direct evaluation, the evaluated potentials must be unpermuted as a post processing step

In [5]:
# Examine unsorted potentials
fmm.all_potentials_u

# Direct evaluation
direct = np.zeros(n_targets * eval_type.value).astype(dtype)
fmm.evaluate_kernel(EvalType.Value, sources, targets, charges, direct)

assert np.allclose(fmm.all_potentials_u[0][:, 0], direct)

#### Clear Charge Data

We offer the option to clear associated charge/potential data and re-run the FMM for the same set of points with new charge data.

In [6]:
charges = np.random.rand(n_sources * n_vec).astype(dtype)
fmm.clear(charges)  # Can be re-evaluated

fmm.evaluate()

# Direct evaluation
direct = np.zeros(n_targets * eval_type.value).astype(dtype)
fmm.evaluate_kernel(EvalType.Value, sources, targets, charges, direct)

assert np.allclose(fmm.all_potentials_u[0][:, 0], direct)

## 3D Plot with MayaVi

In [7]:
from kifmm_py import read_stl_triangle_mesh_vertices

#### Read example mesh from a file

In [8]:
(sources, faces) = read_stl_triangle_mesh_vertices("example.stl")
x = sources[:, 0]
y = sources[:, 1]
z = sources[:, 2]
sources = sources.flatten()

#### Plot Data

In [9]:
mlab.view(azimuth=40, elevation=70, distance="auto", focalpoint="auto")
fig = mlab.figure(
    size=(960, 1080), bgcolor=(1, 1, 1)
)  # This sets the window size for rendering
plot = mlab.triangular_mesh(
    x, y, z, faces, color=(0.5, 0.5, 0.5), representation="surface", figure=fig
)
mlab.show()

#### Setup FMM and Evaluate

In [10]:
dim = 3
dtype = np.float32

# Set FMM Parameters
expansion_order = np.array([6])
n_vec = 1
n_crit = 150
n_sources = len(sources) // dim

# Set Random charges
charges = np.random.rand(n_sources).astype(dtype)

kernel = LaplaceKernel(dtype, EvalType.Value)

field_translation = FftFieldTranslation(kernel, block_size=32)
tree = SingleNodeTree(sources, sources, charges, n_crit=n_crit, prune_empty=True)

# Create FMM runtime object
fmm = KiFmm(expansion_order, tree, field_translation, timed=True)

fmm.evaluate()

#### Plot Solution

In [11]:
mlab.view(azimuth=40, elevation=70, distance="auto", focalpoint="auto")
fig = mlab.figure(
    size=(960, 1080), bgcolor=(1, 1, 1)
)  # This sets the window size for rendering
solution = mlab.triangular_mesh(
    x,
    y,
    z,
    faces,
    scalars=np.log(fmm.all_potentials_u).flatten(),
    representation="surface",
)

# More Complex Examples

### 1) FMM With Multiple Charge Vectors

Here we evaluate multiple charge vectors for the Laplace FMM, with the same set of source/target points. In this case `BlasFieldTranslation`s must be used for the M2L field translation. Additionally we evaluate `EvalType.ValueDeriv`, computing both the potentials and potential derivatives.

In [12]:
np.random.seed(0)

dim = 3
dtype = np.float32

# Set FMM Parameters
expansion_order = np.array([6], np.uint64)  # Single expansion order as using n_crit
n_vec = 5
n_crit = 150
n_sources = 1000
n_targets = 2000

# Setup source/target/charge data in Fortran order
sources = np.random.rand(n_sources * dim).astype(dtype)
targets = np.random.rand(n_targets * dim).astype(dtype)
charges = np.random.rand(n_sources * n_vec).astype(dtype)

svd_threshold = 1e-7

eval_type = EvalType.ValueDeriv

# EvalType computes either potentials (EvalType.Value) or potentials + derivatives (EvalType.ValueDeriv)
kernel = LaplaceKernel(dtype, eval_type)

tree = SingleNodeTree(sources, targets, charges, n_crit=n_crit, prune_empty=True)

try:
    # Will raise TypeError
    field_translation = FftFieldTranslation(kernel, block_size=32)
    # Create FMM runtime object
    fmm = KiFmm(expansion_order, tree, field_translation, timed=True)
except TypeError as e:
    print(e)
    field_translation = BlasFieldTranslation(kernel, svd_threshold, random=False)
    # Create FMM runtime object
    fmm = KiFmm(expansion_order, tree, field_translation, timed=True)


# Run FMM
fmm.evaluate()

# Check wrt to direct evaluation

leaf = fmm.leaves_target_tree[0]
leaf_potentials = fmm.leaf_potentials(leaf)
leaf_targets = fmm.coordinates_target_tree(leaf)
n_targets = len(leaf_targets) // 3

for i, evaluated in enumerate(leaf_potentials):
    direct = np.zeros(n_targets * eval_type.value).astype(dtype)
    fmm.evaluate_kernel(
        eval_type,
        fmm._tree.sources,
        leaf_targets,
        fmm._tree.charges[i * n_sources : (i + 1) * n_sources],
        direct,
    )
    assert np.allclose(evaluated, direct, 1e-2)

Multiple charge vectors only supported with BlasFieldTranslation


### 2) Helmholtz FMM

Support for the low frequency Helmholtz FMM is an active area of development, there are currently no (speed) performance guarantees, and the API is offered as is.

In [13]:
np.random.seed(0)

dim = 3
dtype = np.float32
ctype = np.complex64

# Set FMM Parameters
expansion_order = np.array([6], np.uint64)  # Single expansion order as using n_crit
n_vec = 1
n_crit = 150
n_sources = 1000
n_targets = 2000
wavenumber = dtype(10)
prune_empty = True

# Setup source/target/charge data in Fortran order
sources = np.random.rand(n_sources * dim).astype(dtype)
targets = np.random.rand(n_targets * dim).astype(dtype)
charges = np.random.rand(n_sources * n_vec).astype(ctype)

eval_type = EvalType.Value

# EvalType computes either potentials (EvalType.Value) or potentials + derivatives (EvalType.ValueDeriv)
kernel = HelmholtzKernel(dtype, wavenumber, eval_type)

tree = SingleNodeTree(sources, targets, charges, n_crit=n_crit, prune_empty=prune_empty)

field_translation = FftFieldTranslation(kernel, block_size=32)

# Create FMM runtime object
fmm = KiFmm(expansion_order, tree, field_translation, timed=True)

# Evaluate FMM
fmm.evaluate()

# Direct evaluation check
direct = np.zeros(n_targets * eval_type.value).astype(ctype)
fmm.evaluate_kernel(EvalType.Value, sources, targets, charges, direct)

# assert np.allclose(fmm.all_potentials_u, direct)
assert np.allclose(np.abs(direct), np.abs(fmm.all_potentials_u[0].T), 1e-3)